In [34]:
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
# ops 
from time import time
import time
import math
import os 
import pandas as pd 
import numpy as np 



In [2]:
# import sc  (SparkContext)
from pyspark import SparkContext
sc =SparkContext()

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
datasets_path = '/Users/yennanliu/movie_recommendation/datasets/'
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
df_rating=pd.read_csv(small_ratings_file)
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
df_movie=pd.read_csv(small_movies_file)

In [5]:
df_rating.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [6]:
df_movie.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [185]:
import time 


def get_data_preview():
	datasets_path = '/Users/yennanliu/movie_recommendation/datasets/'
	small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
	small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
	print (" ----------------------- ")
	time.sleep(5)
	df_rating=pd.read_csv(small_ratings_file)
	df_movie=pd.read_csv(small_movies_file)
	print (df_rating.head(3))
	print (df_movie.head(3))
	print (" ----------------------- ")
	time.sleep(5)

    
def get_movie_name_from_id(movie_title,id):
	movie_list = dict(movie_title.take(10000))
	try:
		movie_name = movie_list[id]
		print ('movie_id :' , id)
		print ('movie_name :' , movie_name)
		return movie_name
	except Exception as e:
		#print (e)
		print ('insert movie_id :' , id)
		print ('movie_id not exist')







def get_data(full_dataset=False):

	datasets_path = '/Users/yennanliu/movie_recommendation/datasets/'
	if full_dataset==False:
		#------  rating small dataset  ------#
		# userid, movieid, rating, timestamp 
		small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
		small_ratings_raw_data = sc.textFile(small_ratings_file)
		# get heater 
		small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
		# filter out header 
		# only get 1st, 2rd, and 3rd columns
		# fix dtype to float  
		small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
		.map(lambda line: line.split(",")).map(lambda tokens: (float(tokens[0]),float(tokens[1]),float(tokens[2]))).cache()
		#------ movie dataset ------#
		# movieid, name 
		small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
		small_movies_raw_data = sc.textFile(small_movies_file)
		small_movies_raw_data_header = small_movies_raw_data.take(1)[0]
		# filter out header 
		# fix dtype to float  
		small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
		.map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
		small_movies_titles = small_movies_data.map(lambda x: (int(x[0]),x[1]))
		return small_ratings_data, small_movies_data, small_movies_titles

	elif full_dataset==True:
		#------ rating completed dataset  ------# 
		complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
		complete_ratings_raw_data = sc.textFile(complete_ratings_file)
		complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]
		# filter out header 
		complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
		    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
		#------ movie completed dataset  ------# 
		complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
		complete_movies_raw_data = sc.textFile(complete_movies_file)
		complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]
		# filter out header 
		complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
		    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()
		complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
		return complete_ratings_data, complete_movies_data, complete_movies_titles

 


def get_feature(movie_RDD):
    # get avg / count / features
    small_movie_ID_with_ratings_RDD = (movie_RDD.map(lambda x: (x[1], x[2])).groupByKey())
    small_movie_ID_avg_ratings_RDD = small_movie_ID_with_ratings_RDD.map(get_counts_and_averages)
    small_movie_rating_counts_RDD = small_movie_ID_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))
    print (small_movie_rating_counts_RDD)
    return small_movie_rating_counts_RDD 




def train_test_split(dataset):
	# split data into train (60%), validate (20%), and test (20%)
	training_RDD, validation_RDD, test_RDD = dataset.randomSplit([6, 2, 2])
	validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
	test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))
	return training_RDD, validation_RDD, test_RDD, validation_for_predict_RDD, test_for_predict_RDD


def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

def get_new_input_data():
	new_user_ID = 0

	# The format of each line is (userID, movieID, rating)
	new_user_ratings = [
	 (0,260,9), # Star Wars (1977)
	 (0,1,8), # Toy Story (1995)
	 (0,16,7), # Casino (1995)
	 (0,25,8), # Leaving Las Vegas (1995)
	 (0,32,9), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
	 (0,335,4), # Flintstones, The (1994)
	 (0,379,3), # Timecop (1994)
	 (0,296,7), # Pulp Fiction (1994)
	 (0,858,10) , # Godfather, The (1972)
	 (0,50,8) # Usual Suspects, The (1995)
	]
	new_user_ratings_RDD = sc.parallelize(new_user_ratings)
	print ('New user ratings: %s' % new_user_ratings_RDD.take(10))
	return new_user_ratings_RDD, new_user_ratings,new_user_ID




def get_user_input_data(userid,movieid_array,rating_array):
	new_user_ID = userid
	new_user_ID_array=[new_user_ID for x in range(len(movieid_array))]
	# The format of each line is (userID, movieID, rating)
	new_user_ratings = list(zip(new_user_ID_array,movieid_array,rating_array))
	new_user_ratings_RDD = sc.parallelize(new_user_ratings)
	print ('New user ratings: %s' % new_user_ratings_RDD.take(10))
	return new_user_ratings_RDD, new_user_ratings,new_user_ID




# ML 


def ALS_model(training_RDD,validation_RDD,validation_for_predict_RDD):
	# ------------- 
	# super parameters
	err=0
	min_error = float('inf')
	parameter = {}
	parameter['seed'] = 30
	parameter['iterations'] = 10
	parameter['regularization_parameter'] = 0.1
	parameter['ranks'] = [4, 8, 12]
	parameter['errors'] = [0, 0, 0]
	parameter['tolerance'] = 0.02
	# -------------
	# train the model over super parameters sets 
	for rank in parameter['ranks']:
		model = ALS.train(training_RDD, rank, seed=parameter['seed'], iterations=parameter['iterations'],lambda_=parameter['regularization_parameter'])
		predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
		# join real rating and predicted rating 
		rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
		# predicted rating error (mean square error)
		error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
		parameter['errors'][err] = error
		err += 1
		# --- fix here for python 3 --- #
		print ('For rank %s the RMSE is %s' % (rank, error))
		if error < min_error:
			min_error = error
			best_rank = rank
	# --- fix here for python 3 --- #
	print ('The best model was trained with rank %s' % best_rank)

	return model, predictions, rates_and_preds, min_error,best_rank, parameter


def ALS_model_predict(model,test_for_predict_RDD,test_RDD):
	#model = ALS.train(training_RDD, best_rank, seed=parameter['seed'], iterations=parameter['iterations'],lambda_=parameter['regularization_parameter'])
	predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
	rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
	error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
	print ('For testing data the RMSE is %s' % (error))



In [8]:
########################## run process step by step  ##########################

In [9]:
# get data 
small_ratings_data,small_movies_data,small_movies_titles = get_data()

In [10]:
# train, test split 
training_RDD, validation_RDD, test_RDD, validation_for_predict_RDD, test_for_predict_RDD = train_test_split(small_ratings_data)

In [11]:
# run ALS model 

model, predictions, rates_and_preds, errors, min_error,best_rank = ALS_model(training_RDD,validation_RDD,validation_for_predict_RDD)

For rank 4 the RMSE is 0.9455515910520641
For rank 8 the RMSE is 0.9472441210112387
For rank 12 the RMSE is 0.951113582929365
The best model was trained with rank 4


In [12]:
# preview rating data 
small_ratings_data.take(3)

[(1.0, 31.0, 2.5), (1.0, 1029.0, 3.0), (1.0, 1061.0, 3.0)]

In [13]:
# preview movie data 
small_movies_data.take(3)

[('1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 ('2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 ('3', 'Grumpier Old Men (1995)', 'Comedy|Romance')]

In [14]:
# preview predict 
# userid, movieid, rating
predictions.take(3)

[((80, 1084), 4.3278394674388565),
 ((452, 1084), 3.330589866564371),
 ((65, 1084), 4.0599194778091805)]

In [15]:
###  preview ture rating and predictions ### 
# userid, movieid, user rating, predicted rating
rates_and_preds.take(3)

[((563, 44191), (4.0, 3.8801663012643)),
 ((624, 688), (2.0, 1.9910253946350482)),
 ((378, 8376), (3.5, 1.552039272884325))]

In [16]:
# view errors 
errors

0.9455515910520641

In [17]:
# view min errors (best output)
# view best rank (super parameter set :  ranks = [4, 8, 12])
min_error ,best_rank

(4,
 {'errors': [0.9455515910520641, 0.9472441210112387, 0.951113582929365],
  'iterations': 10,
  'ranks': [4, 8, 12],
  'regularization_parameter': 0.1,
  'seed': 30,
  'tolerance': 0.02})

In [18]:
parameter = {}
parameter['seed'] = 30
parameter['iterations'] = 10
parameter['regularization_parameter'] = 0.1
parameter['ranks'] = [4, 8, 12]
parameter['errors'] = [0, 0, 0]
parameter['tolerance'] = 0.02

In [19]:
# get avg / count / features 

#movie_ID_with_ratings_RDD
#movie_ID_with_avg_ratings_RDD
#movie_rating_counts_RDD

small_movie_ID_with_ratings_RDD = (small_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
small_movie_ID_avg_ratings_RDD = small_movie_ID_with_ratings_RDD.map(get_counts_and_averages)

small_movie_rating_counts_RDD = small_movie_ID_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [186]:
#small_movie_rating_counts_RDD.take(10)

In [21]:
### create user comment (new data)

new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,9), # Star Wars (1977)
     (0,1,8), # Toy Story (1995)
     (0,16,7), # Casino (1995)
     (0,25,8), # Leaving Las Vegas (1995)
     (0,32,9), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,4), # Flintstones, The (1994)
     (0,379,3), # Timecop (1994)
     (0,296,7), # Pulp Fiction (1994)
     (0,858,10) , # Godfather, The (1972)
     (0,50,8) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: %s' % new_user_ratings_RDD.take(10))

New user ratings: [(0, 260, 9), (0, 1, 8), (0, 16, 7), (0, 25, 8), (0, 32, 9), (0, 335, 4), (0, 379, 3), (0, 296, 7), (0, 858, 10), (0, 50, 8)]


In [22]:
# add "New user ratings" to  small_ratings_data
small_ratings_data_with_new_ratings_RDD = small_ratings_data.union(new_user_ratings_RDD)


In [24]:

seed = 30
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

In [37]:

from time import time

t0 = time()
new_ratings_model = ALS.train(small_ratings_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ("New model trained in %s seconds" % round(tt,3))


New model trained in 4.192 seconds


In [38]:
new_ratings_model

In [39]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (small_ratings_data_with_new_ratings_RDD.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(small_movie_rating_counts_RDD)



In [41]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(small_movies_titles).join(small_movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)



[(16, ((3.925905239758646, 'Casino (1995)'), 88)),
 (16, ((4.3375719299637545, 'Casino (1995)'), 88)),
 (16, ((2.7234310871261815, 'Casino (1995)'), 88))]

In [44]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
('Four Rooms (1995)', 4.962593477238005, 26)
('Toy Story (1995)', 4.684409274730289, 247)
('Sense and Sensibility (1995)', 4.594700692133663, 86)
('Heat (1995)', 4.5856974168982605, 104)
('Toy Story (1995)', 4.526296826793137, 247)
('Father of the Bride Part II (1995)', 4.503322125116864, 56)
('Toy Story (1995)', 4.489685055421996, 247)
('Heat (1995)', 4.414854405930663, 104)
('Toy Story (1995)', 4.3860048054231235, 247)
('Sense and Sensibility (1995)', 4.3764159874808835, 86)
('Toy Story (1995)', 4.375264095624171, 247)
('"City of Lost Children', 4.3719383586166884, 40)
('Jumanji (1995)', 4.366295872814058, 107)
('Toy Story (1995)', 4.354807053314385, 247)
('Casino (1995)', 4.343167892672357, 88)
('Casino (1995)', 4.3375719299637545, 88)
('Casino (1995)', 4.33085161463724, 88)
('"Usual Suspects', 4.30273024011923, 201)
("Mr. Holland's Opus (1995)", 4.298549528710479, 87)
('Toy Story (1995)', 4.2965771675293, 247)
('"American Presiden

In [188]:
########################## run the whole training process  ##########################


In [105]:

#get_data_preview()
# get data 
small_ratings_data, small_movies_data, small_movies_titles = get_data()
# train, test split 
training_RDD, validation_RDD, test_RDD, validation_for_predict_RDD, test_for_predict_RDD = train_test_split(small_ratings_data)
# ------------ Model Training  ------------ #
# train ALS model 
model, predictions, rates_and_preds, min_error,best_rank, parameter = ALS_model(training_RDD,validation_RDD,validation_for_predict_RDD)
# predict with trained ALS model 
print ('************')
ALS_model_predict(model,test_for_predict_RDD,test_RDD)
print ('************')



#### train with new input data ###
# get avg / count / features
small_movie_ID_with_ratings_RDD = (small_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
small_movie_ID_avg_ratings_RDD = small_movie_ID_with_ratings_RDD.map(get_counts_and_averages)
small_movie_rating_counts_RDD = small_movie_ID_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))
print (small_movie_rating_counts_RDD.take(3))
# add "New user ratings" to small_ratings_data
new_user_ratings_RDD, new_user_ratings,new_user_ID = get_new_input_data()
small_ratings_data_with_new_ratings_RDD = small_ratings_data.union(new_user_ratings_RDD)
print (small_ratings_data_with_new_ratings_RDD.take(10))
# re-train the model with merged data 
new_ratings_model = ALS.train(small_ratings_data_with_new_ratings_RDD, best_rank, seed=parameter['seed'], 
                          iterations=parameter['iterations'], lambda_=parameter['regularization_parameter'])
print (new_ratings_model)
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (small_ratings_data_with_new_ratings_RDD.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
#new_user_recommendations_RDD = new_ratings_model.predictAll(small_movie_rating_counts_RDD)
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
print ('=======================')
print (new_user_recommendations_RDD.distinct().take(10)) # remove duplicate recommended movie ids 
print ('=======================')
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(small_movies_titles).join(small_movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD = \
new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))
# remove duplicate recommended movie ids 
top_movies = new_user_recommendations_rating_title_and_count_RDD.distinct().filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])
print ('=======================')
print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))
print ('=======================')

For rank 4 the RMSE is 0.9408799307255916
For rank 8 the RMSE is 0.9475998032664866
For rank 12 the RMSE is 0.9496789864812137
The best model was trained with rank 4
************
For testing data the RMSE is 0.948721758451241
************
[(2.0, 107), (97328.0, 1), (4.0, 13)]
New user ratings: [(0, 260, 9), (0, 1, 8), (0, 16, 7), (0, 25, 8), (0, 32, 9), (0, 335, 4), (0, 379, 3), (0, 296, 7), (0, 858, 10), (0, 50, 8)]
[(1.0, 31.0, 2.5), (1.0, 1029.0, 3.0), (1.0, 1061.0, 3.0), (1.0, 1129.0, 2.0), (1.0, 1172.0, 4.0), (1.0, 1263.0, 2.0), (1.0, 1287.0, 2.0), (1.0, 1293.0, 2.0), (1.0, 1339.0, 3.5), (1.0, 1343.0, 2.0)]
[Rating(user=0, product=496, rating=5.916224600729824), Rating(user=0, product=349, rating=6.165838514210409), Rating(user=0, product=353, rating=6.1812863730122825), Rating(user=0, product=271, rating=4.8335955795266425), Rating(user=0, product=335, rating=4.15584619159136), Rating(user=0, product=312, rating=5.773801618133824), Rating(user=0, product=96, rating=1.667535437846